## 模型量化的简单实现
主要包括后量化（PTQ）和感知量化（QAT）  
作者：genggng  日期：2022年6月29日

### 1. 后量化的实现

量化的讲实数转为低比特的整数，转换公式为：
$$r= S(q-Z)$$
$$q=round(\frac{r}{S}+Z)$$
后量化的关键就是计算出scale（实数和整数的放缩比例）和zero point（实数0量化后对应的整数）.
$$S=\frac{r_{\max }-r_{\min }}{q_{\max }-q_{\min }}$$
$$ Z = round(q_{\max}-\frac{r_{max}}{S})$$
下面使用代码实现这两部分完成基本的tensor量化
当出现$Z>q_{\max}$或$Z<q_{\min}$ 时，需要对Z进行截断（因为Z也是用uint存储的)。
此时推导可知$r_{\max}<0$ 或 $r_{\min}>0$ ，因此应该**尽量避免tensor全为正数或者负数的情况**。

In [3]:
import torch
import torch.nn as nn

#### 基本量化操作的实现

In [11]:
def getScaleZeroPoint(min_val,max_val,num_bits=8):
    """
    计算量化参数scale和zero point
    @param
        min_val: 实数最大值
        max_val:  实数最小值
        num_bits:  量化位数
    
    @return
        scale: 实数整数放缩比例
        zero_point: 量化后的零点
    """
    #注意这里输入的mix_val，max_val是标量。
    q_min = 0.
    q_max = 2. ** num_bits - 1
    """
    这里主要是用到qmax和qmin的差值。
    实数和量化数的范围比例为scale，
    rmax放缩后的数和qmax差值就是zero point.
    所以q_min和q_max本身数值并不重要。
    """   
    scale = float(max_val-min_val) / (q_max-q_min)
    zero_point = q_max - max_val/scale

    #为什么要截断zero_point?,因为零点也是用uint8存储的。
    if zero_point < q_min:
        zero_point = torch.tensor([q_min], dtype=torch.float32).to(min_val.device)
    elif zero_point > q_max:
        # zero_point = qmax
        zero_point = torch.tensor([q_max], dtype=torch.float32).to(max_val.device)
    
    zero_point.round_()

    return scale,zero_point

def quantize_tensor(x,scale,zero_point,num_bits=8,signed=False):
    """
    对张量x进行量化
    @param:
        x:待量化浮点数张量
        scale,zero_point:量化参数
        num_bits:量化位数
        signed:采用有符号量化
    @return:
        q_x:量化为整数的张量
    """
    if signed: #量化到有符号数[-128,127]
        q_min = - 2. ** (num_bits-1)
        q_max = 2. ** (num_bits-1) - 1
    else:  #量化到无符号数[0,255]
        q_min = 0.
        q_max = 2. ** num_bits - 1 
    
    q_x = x/scale + zero_point
    q_x.clamp_(q_min,q_max).round_() #使用pytorch内置函数进行截断和四舍五入取整。这一行相当于公式round函数

    return q_x

def dequantize_tensor(q_x,scale,zero_point):
    """
    将量化后的张量q_x反量化为浮点张量x
    @param:
        q_x:量化为整数的张量
        scale,zero_point:量化参数
    return:
        量化之前的浮点张量x
    """
    return scale * (q_x - zero_point) 

In [12]:
x = torch.tensor([-10.0,20.1,23.4,0.1,13.3])
scale,zero_point = getScaleZeroPoint(x.min(),x.max(),8)
q_x = quantize_tensor(x,scale,zero_point)
deq_x = dequantize_tensor(q_x,scale,zero_point)

print("scale={:.3f},zero_point={}".format(scale,zero_point))
print("q_x={}".format(q_x))
print("deq_x={}".format(deq_x))
print("error={}".format(deq_x-x))


scale=0.131,zero_point=76.0
q_x=tensor([  0., 229., 255.,  77., 178.])
deq_x=tensor([-9.9545, 20.0400, 23.4455,  0.1310, 13.3600])
error=tensor([ 0.0455, -0.0600,  0.0455,  0.0310,  0.0600])


#### 量化参数类的实现

我们在量化过程中，需要统计权重和激活值张量的max-min信息，并计算对应的scale和zero point，从而执行量化操作。  
我们可以将要保存的参数和要使用的量化操作封装为一个类，即量化参数。  
量化算法的关键也是量化参数的确定。

In [6]:
class QParam:
    # 就是将上面的代码进行了封装

    def __init__(self,num_bits=8):
        self.num_bits = num_bits
        self.scale = None
        self.zero_point = None
        self.min = None
        self.max = None
        
    def update(self,tensor):
        # 对于输入的待量化张量，更新对应的量化参数
        if self.max is None or self.max <tensor.max():
            self.max = tensor.max()
        self.max = 0 if self.max < 0 else self.max   #保证self.max大于等于0

        if self.min is None or self.min >tensor.min():
            self.min = tensor.min()
        self.min = 0 if self.min > 0 else self.min   #保证self.min小于等于0

        self.scale,self.zero_point = getScaleZeroPoint(min_val=tensor.min(),max_val=tensor.max(),num_bits=8)
    
    def quantize_tensor(self,tensor):
        return quantize_tensor(tensor,self.scale,self.zero_point,self.num_bits)
        
    def quantize_tensor(self,q_x):
        return dequantize_tensor(q_x,self.scale,self.zero_point)


In [7]:
q_parm = QParam(num_bits=8)
x = torch.tensor([-10.0,20.1,23.4,0.1,13.3])
y = torch.tensor([20,20,100,-23,23,30,10,-45])
q_parm.update(x)
q_parm.update(y)
print("q_x=",q_parm.quantize_tensor(x))
print("q_y=",q_parm.quantize_tensor(y))

q_x= tensor([-50.6078, -33.4922, -31.6157, -44.8647, -37.3588])
q_y= tensor([-33.5490, -33.5490,  11.9412, -58.0000, -31.8431, -27.8627, -39.2353,
        -70.5098])


#### 量化网络模块

上面我们能够实现对一个tensor进行量化，但仅仅实现了数据层面上的量化。  
我们还需要对神经网络的模块和运算进行量化，设置适用于量化的网络层。（conv,relu,maxpooling,fc等）


假设卷积的权重 weight 为 w，bias 为 b，输入为 x，输出的激活值为 a。由于卷积本质上就是矩阵运算，因此可以表示成:
$$ a=\sum_{i}^{N} w_{i} x_{i}+b$$  
量化公式为：
$$ S_{a}\left(q_{a}-Z_{a}\right)=\sum_{i}^{N} S_{w}\left(q_{w}-Z_{w}\right) S_{x}\left(q_{x}-Z_{x}\right)+S_{b}\left(q_{b}-Z_{b}\right)$$
$$ q_{a}=\frac{S_{w} S_{x}}{S_{a}} \sum_{i}^{N}\left(q_{w}-Z_{w}\right)\left(q_{x}-Z_{x}\right)+\frac{S_{b}}{S_{a}}\left(q_{b}-Z_{b}\right)+Z_{a}$$
其中令 $M=\frac{S_{w} S_{x}}{S_{a}}$ ,一般让$Z_{b}=0$则
$$q_{a} = M\left(\sum_{i}^{N} q_{w} q_{x}-\sum_{i}^{N} q_{w} Z_{x}-\sum_{i}^{N} q_{x} Z_{w}+\sum_{i}^{N} Z_{w} Z_{x}+q_{b}\right)+Z_{a}$$
从上面可以看出，除了x为动态输入，$q_{w}q_{x}$和$q_{w}Z_{x}$未知，其他的计算结果都可以提前确定下来。  
上面除了M是小数，其他都是整数，并且M可以通过bit shift的方法实现定点乘法。  
因此上式都可以使用整数定点运算完成。

In [8]:
from abc import abstractmethod

class QModule(nn.Module):
    # 创建各种网络模块基类,复用代码
    
    def __init__(self,q_input=True,q_output=True,num_bits=8):
        super().__init__()  #调用父类的构造函数
        """
        网络模块本质是待数据的算子， a = f(x),我们还需要输入x和输出a的量化参数
        但并不是算有模块都有输入，所以需要将上一层的qo作为本层的qi

        q_ipnut:这一层输入的量化参数，包括 S_x,Z_x
        q_output: 这一层输出的量化参数,包括 S_a,Z_a
        """
        self.num_bits = num_bits
        if q_input:
            self.qi = QParam(num_bits=num_bits)
        if q_output:
            self.qo = QParam(num_bits=num_bits)
        
    def freeze(self):
        # 将已经能计算出的静态结果冻结下来，并且由浮点实数转为定点整数
        pass

    @abstractmethod
    def quantize_inference(self,x):
        # 量化推理和正常推理过程不太一致，需要重新编写，因此定义为虚函数。
        raise NotImplemented("quantize_inference should be implemented.")


In [ ]:
class QConv2d(QModule):
    # 二维卷积操作的量化版本
    def __init__(self,conv_module,qi=True,qo=True,num_bits=8):
        # 构造父类的属性
        super().__init__(q_input=qi,q_output=qo,num_bits=num_bits)
        self.conv_module = conv_module    #传入未量化的全精度卷积模块
        self.qw = QParam(num_bits=num_bits)  #卷积层权重的量化参数
    
    def freeze(self,qi=None,qo=None):
        # 为了计算公式中的M，q_w和q_b，并将其

        # 量化卷积层要保证qi和qo都存在，且只被初始化过一次。
        if hasattr(self,'qi') and qi is not None:
            raise ValueError("qi has been provided in init function.")

        if not hasattr(self,'qi') and qi in None:
            raise ValueError("qi is not existed, should be provided.")
        
        if hasattr(self, 'qo') and qo is not None:
            raise ValueError('qo has been provided in init function.')

        if not hasattr(self, 'qo') and qo is None:
            raise ValueError('qo is not existed, should be provided.')
        
        if qi: self.qi = qi
        if qo: self.qo = qo

        # M = S_w*S_x / S_a 
        self.M = self.qw.scale*self.qi.scale / self.qo.scale
        
        # 将卷积核参数q_w 量化为定点整数存储
        self.conv_module.weight.data = self.qw.quantize_tensor(self.conv_module.weight.data) 
        #  ？？？为什么减去zero_point
        self.conv_module.weight.data = self.conv_module.weight.data - self.qw.zero_point

        # 为了方便，使用S_w*S_x来代替S_b
        # 对bias使用对称量化，Z_b=0 (实数中的0和量化后的0相同)
        # 由于卷积运算结果通常使用32bit存储，因此bias也使用32位量化
        self.conv_module.bias.data = quantize_tensor(self.conv_module.bias.data,scale=self.qi.scale*self.qw.scale,zero_point=0,num_bits=32,signed=True)

    def forward(self,x):
        # 伪量化前向推理函数，适用于QAT中反向传播

        if hasattr(self,'qi'):
            self.qi.update(x)  #更新q_x的量化参数

        self.qw.update(self.conv_module.weight.data) #更新q_w的量化参数

        # 进行量化和反量化
        # 实际上是伪量化节点，模拟量化前后的误差
        # 这样的float推理和量化后的int推理具有相同的精度
        self.conv_module.weight.data = self.qw.quantize_tensor(self.conv_module.weight.data)
        self.conv_module.weight.data = self.qw.dequantize_tensor(self.conv_module.weight.data)

        x = self.conv_module(x)

        if hasattr(self,'op'):
            self.qo.update(x) #更新输出q_a的量化参数
        
        return x
    
    def quantize_inference(self,x):
        # 将权重和激活值量化后的推理
        # 因为pytorch平台限制，这里使用float存储整数，进行浮点运算。
        # 实际部署时，应该所有数据和运算都采用定点整数（计算）
        x = x - self.qi.zero_point
        x = self.conv_module(x)
        x = self.M * x + self.qo.zero_point

        return x
       
